<a href="https://colab.research.google.com/github/Parsa33033/ADM_HW3/blob/main/homework3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ADM Homework 3

In [ ]:
from google.colab import drive, files
drive.mount('/content/drive', force_remount = True)

Mounted at /content/drive


In [59]:
import requests
import os
from bs4 import BeautifulSoup
from tqdm import tqdm
from urllib.request import Request, urlopen
import pandas as pd
import csv
import re
import time

## 1. Data collection

In [60]:
num_of_pages = 384
directory = ""
links_file = "anime.csv"

### 1.1. Get the list of animes

In [ ]:
anime = []

                               
if not os.path.isfile(links_file):
    with open(directory + links_file, "w", encoding='utf-8', newline='') as fobj:
        fobj.write("page,name,url\n")
        for page in tqdm(range(0, num_of_pages)):
            url = 'https://myanimelist.net/topanime.php?limit=' + str(page * 50)
            response = requests.get(url)

            soup = BeautifulSoup(response.text, 'html.parser')
            for tag in soup.find_all('tr'):
                links = tag.find_all('a')
                for link in links:        
                    if type(link.get('id')) == str and len(link.contents[0]) > 1:
                        x = (page + 1, link.contents[0], link.get('href'))
                        anime.append(x)
                        writer = csv.writer(fobj)
                        writer.writerow(x)

100%|██████████| 384/384 [02:53<00:00,  2.21it/s]


### 1.2. Crawl animes

In [61]:
pages_dir = directory + "pages"
page_dir = pages_dir + "/" + "page"
article = "article"

In [ ]:
if not os.path.exists(pages_dir):
    os.makedirs(pages_dir)
    
def urlEncodeNonAscii(b):
    return re.sub('[\x80-\xFF]', lambda c: '%%%02x' % ord(c.group(0)), b)

ds = pd.read_csv(directory + links_file)
i = 1
for p, u in tqdm(zip(ds.page, ds.url)):
    d = page_dir + str(p)
    if not os.path.exists(d):
        os.makedirs(d)
    d = page_dir + str(p) + "/" + article + "_" + str(i) + ".html"
    
    if not os.path.exists(d):
#         request = requests.get(u, headers={'Cache-Control': 'no-cache', 'User-Agent': 'Mozilla/5.0'})
        req = Request(u, headers={'User-Agent': 'XYZ/3.0'})
        webpage = urlopen(req, timeout=100).read()
        with open(d, "w", encoding='utf-8') as fobj:
            fobj.write(webpage)
    i += 1

### 1.3 Parse downloaded pages

In [76]:
def getText(parent):
    return ''.join(parent.find_all(text=True, recursive=False)).strip()

In [517]:
def article_parser(article_path, soup):
        anime_title = soup.title.text.strip()
        anime_type = getText(soup.find('span', text="Type:").parent.a)
        anime_num_episodes = getText(soup.find('span', text="Episodes:").parent)
        anime_aired = getText(soup.find('span', text="Aired:").parent).split(" to ")
        anime_status = getText(soup.find('span', text="Status:").parent)
        anime_score = getText(soup.find('span', text="Score:").parent.find_all('span', {'class': 'score-label'})[0])
        anime_users = getText(soup.find('span', text="Members:").parent)
        anime_rank = getText(soup.find('span', text="Ranked:").parent)
        anime_popularity = getText(soup.find('span', text="Popularity:").parent)
        anime_description = getText(soup.find('h2', text="Synopsis").parent.parent.p)
        anime_related = list(set(map(getText,soup.find('h2', text="Related Anime").parent.parent.tr.find_all(lambda tag:tag.name == "a" and tag.href != ""))))
        anime_characters = list(filter(None,list(map(lambda tr: re.split('\n+', tr.find_all('td')[1].text.strip())[0], soup.find('h2', text="Characters & Voice Actors").find_next('div').find_all('tr')))))
        anime_voices = list(filter(None,list(map(lambda tr: re.split('\n+', tr.find_all('td')[0].text.strip())[0], soup.find('h2', text="Characters & Voice Actors").find_next('div').find_all('tr')))))
        anime_staff = list(map(lambda tr: re.split("\n+", tr.text.strip()), soup.find('h2', text="Staff").find_next('div').find_all('tr')))
        l = [anime_title,
                anime_type,
                anime_num_episodes,
                anime_aired,
                anime_status,
                anime_score,
                anime_users,
                anime_rank,
                anime_popularity,
                anime_description,
                anime_related,
                anime_characters,
                anime_voices,
                anime_staff]
        return l
    

In [519]:
num_of_docs = 19125
i = 1
for p in range(1, num_of_pages + 1):
    flag = True
    if (flag): 
        article_path = page_dir + str(p) + "/" + article + "_" + str(i) + ".html"
    while os.path.isfile(article_path):
        article_path = page_dir + str(p) + "/" + article + "_" + str(i) + ".html"
        if os.path.isfile(article_path):
            print(p, "--->", i, "--->", article_path)
            with open(article_path, 'rb') as html:
                soup = BeautifulSoup(html, "html.parser")
                l = article_parser(article_path, soup)
                with open('anime.tsv', 'w', encoding="utf-8") as anime:
                    writer = csv.writer(anime, delimiter='\t')
                    writer.writerow(l)
        i += 1
                


1 ---> 1 ---> pages/page1/article_1.html
1 ---> 2 ---> pages/page1/article_2.html
1 ---> 3 ---> pages/page1/article_3.html
1 ---> 4 ---> pages/page1/article_4.html
1 ---> 5 ---> pages/page1/article_5.html
1 ---> 6 ---> pages/page1/article_6.html
1 ---> 7 ---> pages/page1/article_7.html
1 ---> 8 ---> pages/page1/article_8.html
1 ---> 9 ---> pages/page1/article_9.html
1 ---> 10 ---> pages/page1/article_10.html
1 ---> 11 ---> pages/page1/article_11.html
1 ---> 12 ---> pages/page1/article_12.html
1 ---> 13 ---> pages/page1/article_13.html
1 ---> 14 ---> pages/page1/article_14.html
1 ---> 15 ---> pages/page1/article_15.html
1 ---> 16 ---> pages/page1/article_16.html
1 ---> 17 ---> pages/page1/article_17.html
1 ---> 18 ---> pages/page1/article_18.html
1 ---> 19 ---> pages/page1/article_19.html
1 ---> 20 ---> pages/page1/article_20.html
1 ---> 21 ---> pages/page1/article_21.html
1 ---> 22 ---> pages/page1/article_22.html
1 ---> 23 ---> pages/page1/article_23.html
1 ---> 24 ---> pages/page1/ar

AttributeError: 'NoneType' object has no attribute 'parent'